<!-- HTML file automatically generated from DocOnce source (https://github.com/doconce/doconce/)
doconce format html wall_models_solution.do.txt --ipynb_admon=hrule --without_solutions --no_abort -->
<!-- dom:TITLE: Uncertainty quantification and sensitivity analysis for arterial wall models -->

# Uncertainty quantification and sensitivity analysis for arterial wall models
**Vinzenz Gregor Eck**, Expert Analytics, Oslo  
**Jacob Sturdy**, Department of Structural Engineering, NTNU

Date: **Jun 30, 2021**

In [1]:
# ipython magic
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:

import matplotlib
import matplotlib.pyplot as plt

In [3]:
import chaospy as cp
import chaospy_wrapper as cpw
import numpy as np
from monte_carlo import generate_sample_matrices_mc
from monte_carlo import calculate_sensitivity_indices_mc
unit_mmhg_pa = 133.3
unit_pa_mmhg = 1./unit_mmhg_pa
unit_cm2_m2 = 1. / 100. / 100.
unit_m2_cm2 = 1. / unit_cm2_m2

# Introduction
<div id="sec:introduction"></div>
The arterial wall models we are investigating in this part, are used to describe the (visco-)elastic behaviour of arteries in one-dimensional
simulations of the cardiovascular system.

# Arterial Wall Models
<div id="sec:polynomialChaos"></div>

The elastic wall models are simplified algebraic functions $A(P)$ ([[eck2015stochastic;@boileau_benchmark_2015]](#eck2015stochastic;@boileau_benchmark_2015)),
which state the arterial lumen area $A$ as function of transmural pressure $P$.

For the calibration of the applied wall models, the wave speed in an arterial segment is required.
The wave speed is given from fluid dynamics equations for one-dimensional arteries ([[eck2015stochastic]](#eck2015stochastic)):

<!-- Equation labels as ordinary links -->
<div id="eq:defwaveSpeed"></div>

$$
\begin{equation}
 \tag{1}
c(P) = \sqrt{\frac{A(P)}{\rho\ C(P)}},
\end{equation}
$$

with blood density $\rho= 1050\ [kg\ m^{-3}]$ and compliance $C(P) = dA / dP$.

## *Quadratic* model
The *Quadratic* area-pressure relationship ([[sherwin2003]](#sherwin2003)) is defined as:

<!-- Equation labels as ordinary links -->
<div id="eq:lapArea"></div>

$$
\begin{equation}
A(P) = \left((P-P_s) \frac{A_s}{\lambda} + \sqrt{A_s} \right)^2,
 \tag{2}
\end{equation}
$$

where $\lambda$ is referred to as the stiffness coefficient and $A_s$
is the area at the reference pressure $P_s$.

The stiffness coefficient $\lambda$ is defined as:

<!-- Equation labels as ordinary links -->
<div id="_auto1"></div>

$$
\begin{equation}
\lambda = \frac{2 \rho c_s^2 A_s}{\sqrt{A_s}}
 \tag{3}
\end{equation}
$$

The model is implemented in the following function:

In [4]:
# begin quadratic area model
def quadratic_area_model(pressure_range, samples):
    """
    calculate the arterial vessel wall for a set pressure range
    from 75 to 140 mmHg for a given set of reference wave speeds
    area and pressure.

    :param
        pressure_range: np.array
            pressure range for which to calculate the arterial area

        samples: np.array with shape (4:n_samples)
            sample matrix where
            first indices correspond to
                a_s: reference area
                c_s: reference waves seed
                p_s: reference pressure
                rho: blood density
            and n_samples to the number of samples

    :return:
        arterial pressure : np.array
            of size n_samples
    """
    try:
        pressure_range = pressure_range.reshape(pressure_range.shape[0], 1)
    except AttributeError:
        pass
    a_s, c_s, p_s, rho = samples
    beta = 2*rho*c_s**2/np.sqrt(a_s)*a_s
    #C_Laplace = (2. * ((P - Ps) * As / betaLaplace + np.sqrt(As))) * As / betaLaplace
    return ((pressure_range - p_s)*a_s/beta + np.sqrt(a_s))**2.

## *Logarithmic* model
The *Logarithmic* area-pressure relationship ([[Hayashi1980]](#Hayashi1980)) is defined as:

<!-- Equation labels as ordinary links -->
<div id="eq:expArea"></div>

$$
\begin{equation}
A(P) = A_s \left( 1 + \frac{1}{\beta} ln \left(\frac{P}{P_s}\right) \right)^2,
 \tag{4}
\end{equation}
$$

where $\beta$ is called the stiffness coefficient and $A_s$ is the
area at the reference pressure $P_s$.

The stiffness coefficient $\beta$ is defined as:

<!-- Equation labels as ordinary links -->
<div id="_auto2"></div>

$$
\begin{equation}
\beta = \frac{2\ \rho c_s^2}{P_s}
 \tag{5}
\end{equation}
$$

The model is implemented in the following function:

In [5]:
# begin exponential area model
def logarithmic_area_model(pressure_range, samples):
    """
    calculate the arterial vessel wall for a set pressure range
    from 75 to 140 mmHg for a given set of reference wave speeds
    area and pressure.

    :param
        pressure_range: np.array
            pressure range for which to calculate the arterial area

        samples: np.array with shape (4:n_samples)
            sample matrix where
            first indices correspond to
                a_s: reference area
                c_s: reference waves seed
                p_s: reference pressure
                rho: blood density
            and n_samples to the number of samples

    :return:
        arterial pressure : np.array
            of size n_samples
    """
    try:
        pressure_range = pressure_range.reshape(pressure_range.shape[0], 1)
    except AttributeError:
        pass
    a_s, c_s, p_s, rho = samples
    beta = 2.0*rho*c_s**2./p_s
    #C_hayashi = 2.0 * As / betaHayashi * (1.0 + np.log(P / Ps) / betaHayashi) / P
    return a_s*(1.0 + np.log(pressure_range / p_s)/beta) ** 2.0

## Comparison

For a comparison of the wall models, we set the reference values to:

$$
\begin{align*}
A_s &= 5.12  \;\mathrm{cm^2}      \\
c_s &= 6.256 \;\mathrm{m/s}       \\
P_s &= 100   \;\mathrm{mmHg}      \\
\rho &= 1045 \; \mathrm{kg/m^3}
\end{align*}
$$

The two wall models give almost the same result around the reference area and pressure, for which the model was calibrated.

In [6]:
# start deterministic comparison
pressure_range = np.linspace(45, 180, 100) * unit_mmhg_pa
a_s = 5.12 * unit_cm2_m2
c_s = 6.25609258389
p_s = 100 * unit_mmhg_pa
rho = 1045.

plt.figure()
for model, name in [(quadratic_area_model, 'Quadratic model'), (logarithmic_area_model, 'Logarithmic model')]:
    y_area = model(pressure_range, (a_s, c_s, p_s, rho))
    plt.plot(pressure_range * unit_pa_mmhg, y_area * unit_m2_cm2, label=name)
    plt.xlabel('Pressure [mmHg]')
    plt.ylabel('Area [cm2]')
    plt.legend()
plt.tight_layout()

# Uncertainty Quantification and Sensitivity Analysis
<div id="sec:uqsa"></div>

## Definition of uncertain input parameters
For each model input, we define a uniform distributed random variable which varies with a deviation of $5\%$ around the deterministic value from above:

$$
\begin{align*}
A_s &= 5.12  \; \mathrm{cm^2}    \quad & Z_0 &= \mbox{U}(4.864, 5.376) \\
c_s &= 6.256 \; \mathrm{m/s}     \quad & Z_1 &= \mbox{U}(5.943, 6.569) \\
P_s &= 100   \; \mathrm{mmHg}    \quad & Z_2 &= \mbox{U}(95.0, 105.0) \\
\rho &= 1045 \; \mathrm{kg/m^3}  \quad & Z_3 &= \mbox{U}(992.75, 1097.25) 
\end{align*}
$$

In [7]:
# Create marginal and joint distributions
dev = 0.05
a_s = 5.12 * unit_cm2_m2
A_s = cp.Uniform(a_s * (1. - dev), a_s*(1. + dev))

c_s = 6.25609258389
C_s = cp.Uniform(c_s * (1. - dev), c_s*(1. + dev))

p_s = 100 * unit_mmhg_pa
P_s = cp.Uniform(p_s * (1. - dev), p_s*(1. + dev))

rho = 1045.
Rho = cp.Uniform(rho * (1. - dev), rho*(1. + dev))

jpdf = cp.J(A_s, C_s, P_s, Rho)

## Scatter plots and Samples

In [8]:
# scatter plots
pressure_range = np.linspace(45, 180, 100) * unit_mmhg_pa
Ns = 200
sample_scheme = 'R'
samples = jpdf.sample(Ns, sample_scheme)

for model, name, color in [(quadratic_area_model, 'Quadratic model', '#dd3c30'), (logarithmic_area_model, 'Logarithmic model', '#2775b5')]:
    # evaluate the model for all samples
    Y_area = model(pressure_range, samples)

    plt.figure()
    plt.title('{}: evaluations Y_area'.format(name))
    plt.plot(pressure_range * unit_pa_mmhg, Y_area * unit_m2_cm2)
    plt.xlabel('Pressure [mmHg]')
    plt.ylabel('Area [cm2]')
    plt.ylim([3.5,7.])

    plt.figure()
    plt.title('{}: scatter plots'.format(name))
    for k in range(len(jpdf)):
        plt.subplot(len(jpdf)/2, len(jpdf)/2, k+1)
        plt.plot(samples[k], Y_area[0]*unit_m2_cm2, '.', color=color)
        plt.ylabel('Area [cm2]')
        plt.ylim([3.45, 4.58])
        xlbl = 'Z' + str(k)
        plt.xlabel(xlbl)
    plt.tight_layout()

## Monte Carlo

We apply the Monte Carlo method for both models as discussed before:

In [9]:
# start Monte Carlo
pressure_range = 100 * unit_mmhg_pa
Ns = 5000
sample_method = 'R'
number_of_parameters = len(jpdf)

# 1. Generate sample matrices
A, B, C = generate_sample_matrices_mc(Ns, number_of_parameters, jpdf, sample_method)

ax_dict = {}
figs = {}
for name in ['Quadratic model', 'Logarithmic model'] :
    figs[name] = plt.figure(name)
    ax = ax_dict[name]= figs[name].add_subplot(1,2,1)

print("MC")
for model, name, color in [(quadratic_area_model, 'Quadratic model', '#dd3c30'),
                           (logarithmic_area_model, 'Logarithmic model', '#2775b5')]:
    # evaluate the model for all samples
    Y_A = model(pressure_range, A.T)
    Y_B = model(pressure_range, B.T)

    Y_C = np.empty((number_of_parameters, Ns))
    for i in range(number_of_parameters):
        Y_C[i, : ] = model(pressure_range, C[i, :].T)

    # calculate statistics
    plotMeanConfidenceAlpha = 5.
    Y = np.hstack([Y_A, Y_B])
    expected_value = np.mean(Y, axis=0)
    variance = np.var(Y, axis=0)
    std = np.sqrt(np.var(Y, axis=0))
    prediction_interval = np.percentile(Y, [plotMeanConfidenceAlpha / 2., 100. - plotMeanConfidenceAlpha / 2.], axis=0)*unit_m2_cm2

    print('Mean: {:2.5f} | Std: {:2.5f} : {}'.format(np.mean(expected_value)* unit_m2_cm2, np.mean(std)* unit_m2_cm2, name))

    print('Prediction interval: {:2.5f} | Std: {:2.5f} : {}'.format(prediction_interval[0], prediction_interval[1] , name))
    # 3. Approximate the sensitivity indices
    S, ST = calculate_sensitivity_indices_mc(Y_A, Y_B, Y_C)

    ## Plots
    fig = figs[name]
    ax = ax_dict[name]
    ax.set_title('{}: sensitvity indices'.format(name))
    colorsPie = ['yellowgreen', 'gold', 'lightskyblue', 'lightcoral']
    labels = ['Area', 'wave speed', 'pressure', 'density']
    N = 4
    ind = np.arange(N)  # the x locations for the groups
    width = 0.35  # the width of the bars

    rects1 = ax.bar(ind, S, width, color=colorsPie, alpha = 0.5)
    rects2 = ax.bar(ind + width, ST, width, color=colorsPie, hatch = '.')

    # add some text for labels, title and axes ticks
    ax.set_ylabel('Si')
    ax.set_xticks(ind + width)
    ax.set_xticklabels(labels, rotation=45, ha='right')
    ax.set_xlim([-width, N + width])
    ax.set_ylim([0, 1])
    ax.spines['top'].set_visible(False)
    ax.tick_params(axis='x', top='off')
    ax.spines['right'].set_visible(False)
    ax.tick_params(axis='y', right='off')

    fig.tight_layout()

## Polynomial chaos

We apply the Polynomial Chaos method for both models as discussed before:

In [10]:
# start polynomial chaos
# orthogonal C_polynomial from marginals
polynomial_order = 3

ax_dict = {}
for name in ['Quadratic model', 'Logarithmic model'] :
    figs[name] = plt.figure(name)
    ax = ax_dict[name]= figs[name].add_subplot(1,2,2)

print("GPC")
for model, name, color in [(quadratic_area_model, 'Quadratic model', '#dd3c30'),
                           (logarithmic_area_model, 'Logarithmic model', '#2775b5')]:
    sample_scheme = 'R'
    # create orthogonal polynomials
    basis = cpw.generate_basis(polynomial_order, jpdf)
    # create samples
    Ns = 2*len(basis['poly'])
    samples = jpdf.sample(Ns, sample_scheme)
    # evaluate the model for all samples
    Y_area = model(pressure_range, samples)
    # polynomial chaos expansion
    polynomial_expansion = cpw.fit_regression(basis, samples, Y_area)

    # calculate statistics
    plotMeanConfidenceAlpha = 5
    expected_value = cpw.E(polynomial_expansion, jpdf)
    variance = cpw.Var(polynomial_expansion, jpdf)
    standard_deviation = cpw.Std(polynomial_expansion, jpdf)
    prediction_interval = cpw.Perc(polynomial_expansion,
                                  [plotMeanConfidenceAlpha / 2., 100 - plotMeanConfidenceAlpha / 2.],
                                  jpdf)* unit_m2_cm2
    print('Mean: {:2.5f} | Std: {:2.5f} : {}'.format(np.mean(expected_value) * unit_m2_cm2, np.mean(std) * unit_m2_cm2, name))
    print('Prediction interval: {:2.5f} | Std: {:2.5f} : {}'.format(prediction_interval[0], prediction_interval[1] , name))

    # compute sensitivity indices
    S = cpw.Sens_m(polynomial_expansion, jpdf)
    ST = cpw.Sens_t(polynomial_expansion, jpdf)

    fig = figs[name]
    ax = ax_dict[name]
    ax.set_title('{}: sensitvity indices'.format(name))
    colorsPie = ['yellowgreen', 'gold', 'lightskyblue', 'lightcoral']
    labels = ['Area', 'wave speed', 'pressure', 'density']
    N = 4
    ind = np.arange(N)  # the x locations for the groups
    width = 0.35  # the width of the bars


    rects1 = ax.bar(ind, S, width, color=colorsPie, alpha = 0.5)
    rects2 = ax.bar(ind + width, ST, width, color=colorsPie, hatch = '.')

    # add some text for labels, title and axes ticks
    ax.set_ylabel('Si')
    ax.set_xticks(ind + width)
    ax.set_xticklabels(labels, rotation=45, ha='right')
    ax.set_xlim([-width, N + width])
    ax.set_ylim([0, 1])
    ax.spines['top'].set_visible(False)
    ax.tick_params(axis='x', top='off')
    ax.spines['right'].set_visible(False)
    ax.tick_params(axis='y', right='off')

    fig.tight_layout()

## Averaging over a range of pressures
In the lectures and this example we have focused on only scalar outputs, but it is sometimes important to consider the influence of inputs on vectorial or functional outputs. The following code repeats the above analysis for the output vector of areas evaluated at pressures between 80 and 140.

In [11]:
# Clear the figures so new plots are independent
plt.close("all")
!ec python

===== Monte Carlo =====

We apply the Monte Carlo method for both models as discussed before:

!bc pycod
# start vector Monte Carlo
pressure_range = np.linspace(45, 180, 100) * unit_mmhg_pa
Ns = 5000
sample_method = 'R'
number_of_parameters = len(jpdf)

# 1. Generate sample matrices
A, B, C = generate_sample_matrices_mc(Ns, number_of_parameters, jpdf, sample_method)

fig_mean = plt.figure('mean')
ax_mean = fig_mean.add_subplot(1,2,1)
ax_dict = {}
figs = {}
for name in ['Quadratic model', 'Logarithmic model'] :
    figs[name] = plt.figure(name)
    ax = ax_dict[name]= figs[name].add_subplot(1,2,1)
    
print("\n Uncertainty measures (averaged)\n")
print('\n  E(Y)  |  Std(Y) \n')
for model, name, color in [(quadratic_area_model, 'Quadratic model', '#dd3c30'),
                           (logarithmic_area_model, 'Logarithmic model', '#2775b5')]:
    # evaluate the model for all samples
    Y_A = model(pressure_range, A.T).T
    Y_B = model(pressure_range, B.T).T

    Y_C = np.empty((number_of_parameters, Ns, len(pressure_range)))
    for i in range(number_of_parameters):
        Y_C[i, :, : ] = model(pressure_range, C[i, :, :].T).T

    # calculate statistics
    plotMeanConfidenceAlpha = 5.
    Y = np.vstack([Y_A, Y_B])
    expected_value = np.mean(Y, axis=0)
    variance = np.var(Y, axis=0)
    std = np.sqrt(np.var(Y, axis=0))
    prediction_interval = np.percentile(Y, [plotMeanConfidenceAlpha / 2., 100. - plotMeanConfidenceAlpha / 2.], axis=0)

    print('{:2.5f} | {:2.5f} : {}'.format(np.mean(expected_value)* unit_m2_cm2, np.mean(std)* unit_m2_cm2, name))

    # 3. Approximate the sensitivity indices
    S, ST = calculate_sensitivity_indices_mc(Y_A, Y_B, Y_C)

    ## Plots
    fig = fig_mean
    ax_mean = ax_mean
    ax_mean.plot(pressure_range * unit_pa_mmhg, expected_value * unit_m2_cm2, label=name, color=color)
    ax_mean.fill_between(pressure_range * unit_pa_mmhg, prediction_interval[0] * unit_m2_cm2,
                     prediction_interval[1] * unit_m2_cm2, alpha=0.3, color=color)
    ax_mean.set_xlabel('Pressure [mmHg]')
    ax_mean.set_ylabel('Area [cm2]')
    ax_mean.legend()
    fig.tight_layout()

    fig = figs[name]
    ax = ax_dict[name]
    ax.set_title('{}: sensitvity indices'.format(name))
    colorsPie = ['yellowgreen', 'gold', 'lightskyblue', 'lightcoral']
    labels = ['Area', 'wave speed', 'pressure', 'density']
    N = 4
    ind = np.arange(N)  # the x locations for the groups
    width = 0.35  # the width of the bars

    firstOrderIndicesTimeAverage = np.mean(S*variance,axis=1)/np.mean(variance)
    totalIndicesTimeAverage = np.mean(ST*variance,axis=1)/np.mean(variance)
    rects1 = ax.bar(ind, firstOrderIndicesTimeAverage, width, color=colorsPie, alpha = 0.5)
    rects2 = ax.bar(ind + width, totalIndicesTimeAverage, width, color=colorsPie, hatch = '.')

    # add some text for labels, title and axes ticks
    ax.set_ylabel('Si')
    ax.set_xticks(ind + width)
    ax.set_xticklabels(labels, rotation=45, ha='right')
    ax.set_xlim([-width, N + width])
    ax.set_ylim([0, 1])
    ax.spines['top'].set_visible(False)
    ax.tick_params(axis='x', top='off')
    ax.spines['right'].set_visible(False)
    ax.tick_params(axis='y', right='off')

    fig.tight_layout()

## Polynomial chaos

We apply the Polynomial Chaos method for both models as discussed before:

In [12]:
# start vector polynomial chaos
# orthogonal C_polynomial from marginals
polynomial_order = 3

print("\n Uncertainty measures (averaged)\n")
print('\n  E(Y)  |  Std(Y) \n')



ax_mean = fig_mean.add_subplot(1,2,2)
ax_dict = {}
for name in ['Quadratic model', 'Logarithmic model'] :
    figs[name] = plt.figure(name)
    ax = ax_dict[name]= figs[name].add_subplot(1,2,2)
    
for model, name, color in [(quadratic_area_model, 'Quadratic model', '#dd3c30'),
                           (logarithmic_area_model, 'Logarithmic model', '#2775b5')]:
    sample_scheme = 'R'
    # create orthogonal polynomials
    basis = cpw.generate_basis(polynomial_order, jpdf)
    # create samples
    Ns = 2*len(basis['poly'])
    samples = jpdf.sample(Ns, sample_scheme)
    # evaluate the model for all samples
    Y_area = model(pressure_range, samples)
    # polynomial chaos expansion
    polynomial_expansion = cpw.fit_regression(basis, samples, Y_area.T)

    # calculate statistics
    plotMeanConfidenceAlpha = 5
    expected_value = cpw.E(polynomial_expansion, jpdf)
    variance = cpw.Var(polynomial_expansion, jpdf)
    standard_deviation = cpw.Std(polynomial_expansion, jpdf)
    prediction_interval = cpw.Perc(polynomial_expansion,
                                  [plotMeanConfidenceAlpha / 2., 100 - plotMeanConfidenceAlpha / 2.],
                                  jpdf)
    print('{:2.5f} | {:2.5f} : {}'.format(np.mean(expected_value) * unit_m2_cm2, np.mean(std) * unit_m2_cm2, name))

    # compute sensitivity indices
    S = cpw.Sens_m(polynomial_expansion, jpdf)
    ST = cpw.Sens_t(polynomial_expansion, jpdf)

    fig = fig_mean
    ax_mean =  ax_mean
    ax_mean.plot(pressure_range * unit_pa_mmhg, expected_value * unit_m2_cm2, label=name, color=color)
    ax_mean.fill_between(pressure_range * unit_pa_mmhg, prediction_interval[0] * unit_m2_cm2,
                     prediction_interval[1] * unit_m2_cm2, alpha=0.3, color=color)
    ax_mean.set_xlabel('Pressure [mmHg]')
    ax_mean.set_ylabel('Area [cm2]')
    ax_mean.legend()
    fig.tight_layout()

    fig = figs[name]
    ax = ax_dict[name]
    ax.set_title('{}: sensitvity indices'.format(name))
    colorsPie = ['yellowgreen', 'gold', 'lightskyblue', 'lightcoral']
    labels = ['Area', 'wave speed', 'pressure', 'density']
    N = 4
    ind = np.arange(N)  # the x locations for the groups
    width = 0.35  # the width of the bars

    #firstOrderIndicesTimeAverage = S[:, 50]
    #totalIndicesTimeAverage = ST[:, 50]
    #rects1 = plt.bar(ind, firstOrderIndicesTimeAverage, width, color=colorsPie, alpha=0.5)
    #rects2 = plt.bar(ind + width, totalIndicesTimeAverage, width, color=colorsPie, hatch='.')

    firstOrderIndicesTimeAverage = np.mean(S*variance,axis=1)/np.mean(variance)
    totalIndicesTimeAverage = np.mean(ST*variance,axis=1)/np.mean(variance)
    rects1 = ax.bar(ind, firstOrderIndicesTimeAverage, width, color=colorsPie, alpha = 0.5)
    rects2 = ax.bar(ind + width, totalIndicesTimeAverage, width, color=colorsPie, hatch = '.')

    # add some text for labels, title and axes ticks
    ax.set_ylabel('Si')
    ax.set_xticks(ind + width)
    ax.set_xticklabels(labels, rotation=45, ha='right')
    ax.set_xlim([-width, N + width])
    ax.set_ylim([0, 1])
    ax.spines['top'].set_visible(False)
    ax.tick_params(axis='x', top='off')
    ax.spines['right'].set_visible(False)
    ax.tick_params(axis='y', right='off')

    fig.tight_layout()

# References

1. <div id="eck2015stochastic"></div> **V. G. Eck, J. Feinberg, H. P. Langtangen and L. R. Hellevik**.  Stochastic Sensitivity Analysis for Timing and Amplitude of Pressure waves in the Arterial System, *International Journal for Numerical Methods in Biomedical Engineering*, 31(4), 2015.

2. <div id="boileau_benchmark_2015"></div> **E. Boileau, P. Nithiarasu, P. J. Blanco, L. O. Mueller, F. E. Fossan, L. R. Hellevik, W. P. Donders, W. Huberts, M. Willemet and J. Alastruey**.  A benchmark study of numerical schemes for one-dimensional arterial blood flow modelling, *International Journal for Numerical Methods in Biomedical Engineering*, 31(10), pp. n/a-n/a, 2015.

3. <div id="sherwin2003"></div> **S. Sherwin, V. Franke, J. Peir\'o and K. Parker**.  One-Dimensional Modelling of Vascular Network in Space-Time Variables, *Journal of Engineering Mathematics*, 47(3-4), pp. 217-233, 2003.

4. <div id="Hayashi1980"></div> **K. Hayashi, H. Handa, S. Nagasawa, A. Okumura and K. Moritake**.  Stiffness and Elastic Behavior of Human Intracranial and Extracranial arteries, *Journal of Biomechanics*, 13(2), pp. 175-184, 1980.